In [1]:
import sys
import os
import re
import pandas as pd
from collections import defaultdict

RuntimeError: module compiled against API version 0xa but this version of numpy is 0x9

#### JIRA BDD-2328 (SDY739)


1) Multi-sample file :  
SDY739:  
`/home/cris/Desktop/cris/projects/BDD2328/files/Ertl_YO_14.581082.csv`   



In [2]:
##DIRECTORY CELL
beg = 'Please enter file path for '
uarray = input(beg + 'the uarray .csv: \n')
mapping = input(beg + 'gene expression sampleIDs .txt: \n')

Please enter file path for the uarray .csv: 
/home/cris/Desktop/cris/projects/BDD2328/files/Ertl_YO_14.581082.csv
Please enter file path for gene expression sampleIDs .txt: 
"/home/cris/Desktop/cris/projects/BDD2328/files/Ertl_YO_14.581082.csv


In [3]:
study_acc = input("Study account number (like 'SDY739'): \n")
exp_acc = input("experiment account number (like 'EXP14794'): \n")
#uarray = "/home/cris/Desktop/cris/projects/BDD2328/files/Ertl_YO_14.581082.csv"
#mapping = "/home/cris/Desktop/cris/projects/BDD2328/files/sdy739.EXP14794.geneExpression.experimentSampleIDs.accessions.txt"
#df = pd.read_table(mapping)
#df.head()
### Get experiment sample acc mapping to user def accessions
## need to change format Y.001.v1.14 to y4.Y.001.v1 so mapping works with result file

esID = {}
with open(mapping, "r") as mp:
    mp.readline()
    for mpl in mp:
        acc = mpl.split("\t")
        tmp = acc[1].split(".")
        pre = tmp[0]
        if (pre == 'O'):
            pre = 'A'
        userdef_acc = ".".join(["y4", pre, tmp[1], tmp[2]])
        esID[userdef_acc] = acc[0]

Study account number (like 'SDY739'): 
"/home/cris/Desktop/cris/projects/BDD2328/files/Ertl_YO_14.581082.csv
experiment account number (like 'EXP14794'): 
"/home/cris/Desktop/cris/projects/BDD2328/files/Ertl_YO_14.581082.csv


FileNotFoundError: [Errno 2] No such file or directory: '"/home/cris/Desktop/cris/projects/BDD2328/files/Ertl_YO_14.581082.csv'

In [ ]:
## FOR SDY648:
# 1/ Get directory path and create output directory
output_path_file = os.path.split(os.path.realpath(uarray))[0]
sdy_dir = "/".join([output_path_file, study_acc, exp_acc])
os.makedirs(sdy_dir, exist_ok=True)

# 2/ Create new files and get the data to populate them.
sdy_vals = defaultdict(dict)
sdy_probes = {} 
sdy_files = {}
userIDs=[]

with open(uarray, "r") as ua:
    # a. Get the experiment sample acc:
    hdg = ua.readline()
    hdg = hdg.strip()
    hdgs = hdg.split("\t")
    for k in range(4,len(hdgs)): 
        if esID[hdgs[k]].startswith("ES"):
            userIDs.append(esID[hdgs[k]])
    print("processing...")
    
    # b. Create the ouput files -- format will be SDY_ACC_EXP_ACC_EXP_SAM_ACC_NUM_microarray_result.txt
    for esan in set(userIDs):
        new_filename = "_".join([study_acc, exp_acc, esan, "microarray_result.txt"])
        path_file = "/".join([sdy_dir, new_filename])
        sdy_files[esan]= path_file
        
        with open(path_file, "w") as out1:
            out1.write("PROBE_ID\tACC\tNAME\tSYMBOL\tVALUE\n")
    
    print("files created")
    print("getting data...")
    
    secondline = ua.readline()
    # c. Get the data for each expsample
    for lignes in ua:
        lignes = lignes.strip("\n")
        values = lignes.split("\t")

        ## get the probe infos
        if not values[2].strip():
            values[2]="NA"

        if not values[3].strip():
            values[3]="NA"
        
        probeinfo = "\t".join([values[0], values[1], values[2], values[3]])
        sdy_probes[values[0]] = probeinfo
        
        ## get the data for each expsample
        for m in range(4,len(values)) :
            sdy_vals[esID[hdgs[m]]][values[0]] = values[m]
            
            
print("populating files...")
# 3/ Populate the files: 

for exp_samples in sdy_files:
    with open(sdy_files[exp_samples], "a") as singlesplefile:
        for probes in sdy_probes:
            singlesplefile.write("\t".join([sdy_probes[probes], sdy_vals[exp_samples][probes]]))
            singlesplefile.write("\n")
print("done!")


3) Upload to IMMPORT. Upload ticket is .. cthomas_20161130_13487  
4) Run Query 699 with upload ticket.  
5) old file info id: 609663  
6) prepare BCD script for update of the database   

**Uploaded files**  
Result of Query 699  
`/home/cris/Desktop/cris/projects/BDD2328/files/cthomas_699_3799.txt
`

In [ ]:
fnms = "/home/cris/Desktop/cris/projects/BDD2328/files/cthomas_699_3799.txt"
df = pd.read_table(fnms)

df.head()  


In [ ]:
## Check uniqueness of items under FILE_NAME
if len(df.FILE_NAME)!=len(set(df.FILE_NAME)):
    print("there are duplicates in list")

*__ for filenames containing EXP_SAM_ACC_NUM :__*

In [ ]:
## Extract ES number in Query 699 outputs:
def get_exp_sam_acc_num(nms):
    es = re.search("ES\d{7}", nms)
    return es.group()

df["EXP_SAMPLE_ACC_NUM"] = df.FILE_NAME.apply(get_exp_sam_acc_num)
df.head()

In [ ]:
## print the string to use in query 777 containing all EXP_SAM_ACC_NUM to re-link:
tmp=[]
## might need to change EXP_SAM_ACC_NUM and EXP_SAMPLE_ACC_NUM back and forth
for i in set(df.EXP_SAMPLE_ACC_NUM):
    nwstr="'"+i+"'"
    tmp.append(nwstr)
query_text = ",".join(tmp)
print(query_text)

Ran query 777 to check - they all have the same FILE_INFO_ID -- 609663

In [ ]:
## For single sample files from splitting multisample:
fid = "609663"
nwdf = pd.DataFrame({'NEW_FILE_INFO_ID': df.FILE_INFO_ID, 'EXP_SAMPLE_ACC_NUM':df.EXP_SAMPLE_ACC_NUM, 'FILE_INFO_ID':fid})
tmpp = pd.DataFrame({'EXP_SAMPLE_ACC_NUM': nwdf.EXP_SAMPLE_ACC_NUM, 'FILE_INFO_ID': nwdf.FILE_INFO_ID, 'NEW_FILE_INFO_ID': nwdf.NEW_FILE_INFO_ID })
tmpp["DATA_FORMAT"] = "Illumina Microarray"
tmpp["RESULT_SCHEMA"] = "OTHER"

tmpp.head()


BDC scripts:
1. update file_info file purpose and file detail  
`/home/cris/Desktop/cris/projects/BDD2328/bdc/BDC_FILE_INFO_UPD.SDY739_BDD2328.txt`
2. expsample_2_file_info delete old FID + update FID   
`/home/cris/Desktop/cris/projects/BDD2328/bdc/BDC_EXP_SAM_2_FILE_INFO_UPD.SDY739_BDD2328.txt`
3. exp sample data format and result schema  
`/home/cris/Desktop/cris/projects/BDD2328/bdc/BDC_EXPERIMENTAL_SAMPLE_UPD.SDY739_BDD2328.txt`

In [ ]:
output_path_file = os.path.split(os.path.realpath(fnms))[0]
output_path = os.path.split(os.path.realpath(output_path_file))[0]

#NAME OF BDC SCRIPTS
bdc_script_upd_es2fi =  "/".join([output_path, "bdc/BDC_EXP_SAM_2_FILE_INFO_UPD.SDY739_BDD2328.txt"])  
bdc_script_upd_fi =  "/".join([output_path, "bdc/BDC_FILE_INFO_UPD.SDY739_BDD2328.txt"])  
bdc_script_upd_es =  "/".join([output_path, "bdc/BDC_EXPERIMENTAL_SAMPLE_UPD.SDY739_BDD2328.txt"])  

tmp_dir = "/".join([output_path, "tmp"]) 
os.makedirs(tmp_dir, exist_ok=True)

## UPDATE EXP_SAM_2_FILE_INFO
ofile_es2fi = "/".join([output_path, "tmp/bdc_update_exp_sam_2_file.txt"]) 
tmpp.to_csv(ofile_es2fi, sep="\t", header=False, index=False)

with open(bdc_script_upd_es2fi, "w") as bdc_upd_esfi:
    bdc_upd_esfi.write("FORMAT\tUPDATE_LINKAGE\n")
    bdc_upd_esfi.write("TABLE_NAME\tEXP_SAM_2_FILE_INFO\n")
    bdc_upd_esfi.write("KEY_COLUMN\tEXP_SAMPLE_ACC_NUM\n")
    bdc_upd_esfi.write("CHANGING_COLUMN\tFILE_INFO_ID\n")
    bdc_upd_esfi.write("COLUMNS\tDATA_FORMAT~RESULT_SCHEMA\n")
    bdc_upd_esfi.write("DATA STARTED\n")
    with open(ofile_es2fi, "r") as tmp_file1:
        for line in tmp_file1:
            bdc_upd_esfi.write(line)

os.remove(ofile_es2fi)
os.rmdir("/".join([output_path, "tmp"]))

## UPDATE FILE_INFO
with open(bdc_script_upd_fi, "w") as bdc_upd_fi:
    bdc_upd_fi.write("FORMAT\tUPDATE_VAL\n")
    bdc_upd_fi.write("TABLE_NAME\tFILE_INFO\n")
    bdc_upd_fi.write("KEY_COLUMN\tFILE_INFO_ID\n")
    bdc_upd_fi.write("COLUMNS\tFILE_INFO_ID~FILE_DETAIL~FILE_PURPOSE\n")
    bdc_upd_fi.write("DATA STARTED\n")
    for k in set(tmpp.NEW_FILE_INFO_ID):
        bdc_upd_fi.write("\t".join([str(k), "Illumina BeadArray","Gene expression result"]))
        bdc_upd_fi.write("\n")

## UPDATE EXPERIMENTAL_SAMPLE
with open(bdc_script_upd_es, "w") as bdc_upd_es:
    bdc_upd_es.write("FORMAT\tUPDATE_VAL\n")
    bdc_upd_es.write("TABLE_NAME\tEXPERIMENTAL_SAMPLE\n")
    bdc_upd_es.write("KEY_COLUMN\tEXP_SAMPLE_ACC_NUM\n")
    bdc_upd_es.write("COLUMNS\tEXP_SAMPLE_ACC_NUM~EXP_SAMPLE_DATA_FORMAT~EXP_SAMPLE_RESULT_SCHEMA\n")
    bdc_upd_es.write("DATA STARTED\n")
    for l in set(tmpp.EXP_SAMPLE_ACC_NUM):
        bdc_upd_es.write("\t".join([str(l),"Illumina Microarray","OTHER"]))
        bdc_upd_es.write("\n")
